In [1]:
print('Hello')

Hello


In [3]:
import pandas as pd

df = pd.read_csv('../data/train.csv')
df.head()


,content,summary
0,"(nguồn: ngày 123, bộ tài chính nhật bản thừa n...",bộ tài chính nhật bản đã thừa nhận đã chỉnh sử...
1,"ông đỗ thái lâm, phó trưởng ga đà nẵng cho biế...",ga đà nẵng và ga hà nội đã bắt đầu bán vé tàu ...
2,"theo quyết định, phó thủ tướng thường trực ngu...",quyết định của chính phủ đề cập đến việc cử là...
3,những tin tức về táo quân gặp nhau cuối năm 20...,táo quân 2016 đang làm nóng lòng công chúng. x...
4,nga lên kế hoạch đưa người lên mặt trăng và sa...,nga đã lên kế hoạch đưa người lên mặt trăng và...


In [6]:
df[:4].to_csv('text.txt', index=False)

In [7]:
train_df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv('../data/val.csv')
test_df = pd.read_csv('../data/test.csv')
print(len(train_df), len(val_df), len(test_df))

50846 6356 6356


In [11]:
train_df.info()
# val_df.info()
# test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50846 entries, 0 to 50845
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  50846 non-null  object
 1   summary  50846 non-null  object
dtypes: object(2)
memory usage: 794.6+ KB


In [48]:
import os
import pandas as pd

df = pd.read_csv('../data/processed/train.csv')

sample_path = os.path.join("../data/processed/", 'train_raw_sample.txt')
with open(sample_path, 'w', encoding='utf-8') as f:
	f.write("--- MẪU DỮ LIỆU SAU KHI XỬ LÝ ---\n\n")
	sample_df = df[:5]
	for i, row in sample_df.iterrows():
		f.write(f"--- Mẫu {i+1} ---\n")
		f.write(f"Article:\n{row['content']}\n\n")
		f.write(f"Abstract:\n{row['summary']}\n\n")

In [ ]:
import re
import unicodedata

def normalize_dates(text):
    # Thay thế các dấu - . trong ngày tháng thành /
    text = re.sub(r'(\d{1,2})[-.\s](\d{1,2})[-.\s](\d{2,4})', r'\1/\2/\3', text)
    # Xử lý trường hợp chỉ có ngày và tháng
    text = re.sub(r'(\d{1,2})[-.\s](\d{1,2})(?![\d/])', r'\1/\2', text)
    return text

def remove_metadata_lines(text: str) -> str:
    """
    Loại bỏ các dòng meta như chữ ký tác giả, nguồn tin... thường xuất hiện
    ở cuối bài báo (ví dụ: "Hà Phương/VOV.VN.", "Thực hiện: Hiền Nhi - Độc Lập.").
    """
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        stripped = line.strip()
        if not stripped:
            # Bỏ dòng trống
            continue

        # Các dòng bắt đầu bằng cụm từ meta phổ biến
        if re.match(r'^(thực hiện|tác giả|biên soạn|dịch giả)\b', stripped, flags=re.IGNORECASE):
            continue

        # Các dòng chứa tên tòa soạn/báo điện tử dạng "Tên tác giả/VOV.VN"
        if re.search(r'\/vov\.vn|\/vnexpress\.net|\/thanhnienonline|\/thanhnien\.vn|\/tuoitre\.vn', stripped,
                     flags=re.IGNORECASE):
            continue

        # Các dòng ghi dạng "Tên tác giả/TÊN BÁO."
        if re.match(r'^[A-ZÀ-Ỵ][^,]+\/[A-Z0-9.]+\.?$', stripped):
            continue

        cleaned_lines.append(line)

    return "\n".join(cleaned_lines)

def clean_text(text):
    """Hàm xử lý văn bản"""
    

    # Chuẩn hóa Unicode và chuyển văn bản thành chữ thường
    text = unicodedata.normalize("NFC", text)
    text = text.lower().strip()

    # Loại bỏ các dòng meta (tác giả, nguồn tin...) trước
    text = remove_metadata_lines(text)

    # Loại bỏ nhiễu (htmls, urls và kí tự rỗng, chú thích ảnh/video)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', '', text)
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    text = re.sub(r'\s*(ảnh|video)\s*:\s*[\w\s_.-]+', ' ', text)

    # Loại bỏ kí tự đặc biệt
    text = re.sub(r'[^\w\s,.?!:;()/\\-]', '', text)
    text = re.sub(r'\((nguồn|theo).*?\)\s*', '', text)

    # Các bước chuẩn hóa
    text = normalize_dates(text)

    # Thêm khoảng trắng quanh dấu câu, xử lý đặc biệt cho .,/
    text = re.sub(r'\s*([?!:;()\\])\s*', r' \1 ', text)
    text = re.sub(r'((?<!\d)[.,]|[.,](?!\d))', r' \1 ', text)
    text = re.sub(r'(?<!\d)\s*/\s*|\s*/\s*(?!\d)', r' / ', text)

    # Gộp các dấu câu lặp lại (ví dụ: "..." -> ".")
    text = re.sub(r'([.?!,])(\s*\1)+', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [41]:
def normalize_dates(text):
    # Thay thế các dấu - . trong ngày tháng thành /
    text = re.sub(r'(\d{1,2})[-.\s](\d{1,2})[-.\s](\d{2,4})', r'\1/\2/\3', text)
    # Xử lý trường hợp chỉ có ngày và tháng
    text = re.sub(r'(\d{1,2})[-.\s](\d{1,2})(?![\d/])', r'\1/\2', text)
    return text


def remove_metadata_lines(text: str) -> str:
    """
    Loại bỏ các dòng meta như chữ ký tác giả, nguồn tin... thường xuất hiện
    ở cuối bài báo (ví dụ: "Hà Phương/VOV.VN.", "Thực hiện: Hiền Nhi - Độc Lập.").
    """
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        stripped = line.strip()
        if not stripped:
            # Bỏ dòng trống
            continue

        # Các dòng bắt đầu bằng cụm từ meta phổ biến
        if re.match(r'^(thực hiện|tác giả|biên soạn|dịch giả)\b', stripped, flags=re.IGNORECASE):
            continue

        # Các dòng chứa tên tòa soạn/báo điện tử dạng "Tên tác giả/VOV.VN"
        if re.search(r'\/vov\.vn|\/vnexpress\.net|\/thanhnienonline|\/thanhnien\.vn|\/tuoitre\.vn', stripped,
                     flags=re.IGNORECASE):
            continue

        # Các dòng ghi dạng "Tên tác giả/TÊN BÁO."
        if re.match(r'^[A-ZÀ-Ỵ][^,]+\/[A-Z0-9.]+\.?$', stripped):
            continue

        cleaned_lines.append(line)

    return "\n".join(cleaned_lines)

def clean_text(text):
    """Hàm xử lý văn bản"""
    

    # Chuẩn hóa Unicode và chuyển văn bản thành chữ thường
    text = unicodedata.normalize("NFC", text)
    text = text.lower().strip()

    # Loại bỏ các dòng meta (tác giả, nguồn tin...) trước
    text = remove_metadata_lines(text)

    # Loại bỏ nhiễu (htmls, urls và kí tự rỗng, chú thích ảnh/video)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', '', text)
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    # Xóa chú thích ảnh/video nhưng KHÔNG ăn sang dòng tiếp theo
    # (tránh xóa luôn phần đầu câu như "trong một tháng qua" trong ví dụ của bạn)
    text = re.sub(r'\s*(ảnh|video)\s*:\s*[^\n\r]+', ' ', text)

    # Loại bỏ kí tự đặc biệt
    text = re.sub(r'[^\w\s,.?!:;()%/$đ-]', '', text)
    text = re.sub(r'\((nguồn|theo).*?\)\s*', '', text)

    # Các bước chuẩn hóa khác (không động tới ngày tháng, đơn vị để giữ nguyên ngữ cảnh)

    # Thêm khoảng trắng quanh dấu câu (trừ dấu / để không làm đổi định dạng ngày và đơn vị)
    text = re.sub(r'\s*([?!:;()\\])\s*', r' \1 ', text)
    text = re.sub(r'((?<!\d)[.,]|[.,](?!\d))', r' \1 ', text)

    # Gộp các dấu câu lặp lại (ví dụ: "..." -> ".")
    text = re.sub(r'([.?!,])(\s*\1)+', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [45]:
text = """
Nếu đã là fan của vũ trụ Marvel, ắt hẳn bạn sẽ không còn xa lạ gì với Captain Marvel - nữ siêu anh hùng mới nhất, đồng thời là người mạnh nhất biệt đội Avengers. Bước ra từ truyện, cuối cùng thì bộ phim riêng về nữ siêu anh hùngnày đã tung những hình ảnh chính thức đầu tiên.
Captain Marvel Carol Danvers do Brie Larson thủ vai.
Khởi động từ năm 2008, Vũ trụ Điện ảnh Marvel sau một thập kỷ đã chứng kiến nhiều sự thay đổi. Bản thân Captain Marvel cũng là nữ siêu anh hùng đầu tiên sở hữu phim riêng. Dự án điện ảnh dành cho Captain Marvel được manh nha từ cách đây bốn năm, và chỉ đến hôm nay, trailer đầu tiên của tác phẩm mới kịp cập bến để chiều lòng người hâm mộ.
Trong trailer đầu tiên, khán giả được tiết lộ về nguồn gốc của nhân vật Captain Marvel Carol Danvers do Brie Larson thủ vai. Theo đó, với xuất thân là một người lính  , Carol Denvers chưa bao giờquên những ngày tháng phục vụ trong quân đội Mỹ. Nhưng một mặt khác, cô chính là Captain Marvel  cái tên đã nói lên tất cả trọng trách gánh team của một nữ chiến binh quyền uy, được giao nhiệm vụbảo vệ toàn cõi vũ trụ khỏi những thế lực thù địch.
Bối cảnh của phim sẽ diễn ra vào khoảng thập niên 90 của thế kỷ 20. Điều này giải thích vì sao Nick Fury (Samuel L. Jackson) và còn sống và mang một gương mặt trẻ trung hơn. Trong thời kỳ này, vị Giám đốc của S.H.I.E.L.D. đang trên hành trình chiêu mộ những cá nhân có sức mạnh phi thường nằm rải rác trên toàn thế giới.
Ngoài những gương mặt quen thuộc, thì nam tài tử Jude Law cũng có dịp góp mặt trong vai Mar-Vell. Anh thực chất là Captain Marvel tiền nhiệm, đóng vai trò là người hướng dẫn Carol Danvers sử dụng thuần thục nguồn sức mạnh của cả vũ trụ đang cuộn trào bên trong cô.
""Captain Marvel"" dự kiến khởi chiếu vào ngày 8/3/2019./.
Hà Phương/VOV.VN.
"""

In [46]:
x = clean_text(text)

In [47]:
print(x)

nếu đã là fan của vũ trụ marvel , ắt hẳn bạn sẽ không còn xa lạ gì với captain marvel - nữ siêu anh hùng mới nhất , đồng thời là người mạnh nhất biệt đội avengers . bước ra từ truyện , cuối cùng thì bộ phim riêng về nữ siêu anh hùngnày đã tung những hình ảnh chính thức đầu tiên . captain marvel carol danvers do brie larson thủ vai . khởi động từ năm 2008 , vũ trụ điện ảnh marvel sau một thập kỷ đã chứng kiến nhiều sự thay đổi . bản thân captain marvel cũng là nữ siêu anh hùng đầu tiên sở hữu phim riêng . dự án điện ảnh dành cho captain marvel được manh nha từ cách đây bốn năm , và chỉ đến hôm nay , trailer đầu tiên của tác phẩm mới kịp cập bến để chiều lòng người hâm mộ . trong trailer đầu tiên , khán giả được tiết lộ về nguồn gốc của nhân vật captain marvel carol danvers do brie larson thủ vai . theo đó , với xuất thân là một người lính , carol denvers chưa bao giờquên những ngày tháng phục vụ trong quân đội mỹ . nhưng một mặt khác , cô chính là captain marvel cái tên đã nói lên tất c